# Previous LDA Model Results

In [146]:
import pandas as pd
from information_diversity import *

df = pd.read_csv('info_divvy_LDA.csv')

def get_batches(dataframe, batch_size=3):
    batches = []
    rows = list(dataframe.iterrows())
    for i in range(0, len(rows), batch_size):
        batches.append(rows[i:i + batch_size])
    return batches

for i in range(0, 10):
    print("Attempt Number: ", i)
    output_df = get_info_diversity(df, "conversation_num", "message")

    batches = get_batches(output_df)

    count = 0

    # print("ALL RESULTS SHOULD BE OVER ")

    for batch in batches:
        feature = "info_diversity"

        og_result = batch[0][1][feature]
        inv_result = batch[1][1][feature]
        dir_result = batch[2][1][feature]

        inv_distance = abs(og_result - inv_result)
        dir_distance = abs(og_result - dir_result)
        count += 1

        if dir_distance < inv_distance:
            print("[FAILED] Test Case #", batch[0][1]["conversation_num"], ":", dir_distance / inv_distance)
        # print("Test Case #", count, ":", dir_distance / inv_distance)


Attempt Number:  0
[FAILED] Test Case # conversation_7 : 0.7512651685958739
Attempt Number:  1
[FAILED] Test Case # conversation_4 : 0.5894661985718989
[FAILED] Test Case # conversation_7 : 0.34210148136901297
Attempt Number:  2
[FAILED] Test Case # conversation_7 : 0.23791709508394202
Attempt Number:  3
Attempt Number:  4
Attempt Number:  5
[FAILED] Test Case # conversation_4 : 0.004348320675631441
[FAILED] Test Case # conversation_7 : 0.8276635597113645
Attempt Number:  6
[FAILED] Test Case # conversation_4 : 0.09795153483568632
Attempt Number:  7
[FAILED] Test Case # conversation_7 : 0.2380256154675347
Attempt Number:  8
[FAILED] Test Case # conversation_7 : 0.5724496119873153
Attempt Number:  9
[FAILED] Test Case # conversation_4 : 0.6125460013915317
[FAILED] Test Case # conversation_7 : 0.7434659153438612


# Preliminary BERT Model avec Distributions

As we notice, the LDA model trends to be extremely stochastic in its implementation. We are going to pivot to BERTopic to create these topic representations and determine if that's giving us a more fruitful result. We are going to set a parameter to equal the minimum number of chats in a topics to indirectly affect the number of topic generates. With less chats permitted to be assigned a topic, we inherently generate more topics. 

In [56]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired


docs = pd.read_csv('info_divvy.csv')['message'].to_list()
representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=KeyBERTInspired(), min_topic_size=2)
# topics, probs = topic_model.fit_transform(docs)
topics = topic_model.fit(docs)

In [202]:
topics = topic_model.fit(docs)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5,-1_watching_episode_shows_intense,"[watching, episode, shows, intense, same, yeah...",[ I should probably start looking too. I’m jus...
1,0,3,0_internships_applying_assignments_learning,"[internships, applying, assignments, learning,...",[ I am! I’m applying to a few tech companies n...
2,1,2,1_neural_networks_module_tough,"[neural, networks, module, tough, stuck, parts...",[ It’s going well! I’m almost done with the ne...


In [191]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,Did you catch the latest episode of that show...,-1,-1_internships_assignments_learning_intense,"[internships, assignments, learning, intense, ...",[ Yeah it was intense! But I had to pause half...,internships - assignments - learning - intense...,0.0,False
1,Yeah it was intense! But I had to pause halfw...,-1,-1_internships_assignments_learning_intense,"[internships, assignments, learning, intense, ...",[ Yeah it was intense! But I had to pause half...,internships - assignments - learning - intense...,0.0,True
2,I haven’t started watching it yet. Is it real...,-1,-1_internships_assignments_learning_intense,"[internships, assignments, learning, intense, ...",[ Yeah it was intense! But I had to pause half...,internships - assignments - learning - intense...,0.0,False
3,Definitely worth it. But speaking of assignme...,-1,-1_internships_assignments_learning_intense,"[internships, assignments, learning, intense, ...",[ Yeah it was intense! But I had to pause half...,internships - assignments - learning - intense...,0.0,False
4,It’s going well! I’m almost done with the neu...,-1,-1_internships_assignments_learning_intense,"[internships, assignments, learning, intense, ...",[ Yeah it was intense! But I had to pause half...,internships - assignments - learning - intense...,0.0,True
5,Neural networks? That’s tough. I’m still stuc...,-1,-1_internships_assignments_learning_intense,"[internships, assignments, learning, intense, ...",[ Yeah it was intense! But I had to pause half...,internships - assignments - learning - intense...,0.0,False
6,You’ll get there. Just take it one step at a ...,-1,-1_internships_assignments_learning_intense,"[internships, assignments, learning, intense, ...",[ Yeah it was intense! But I had to pause half...,internships - assignments - learning - intense...,0.0,True
7,I am! I’m applying to a few tech companies ne...,-1,-1_internships_assignments_learning_intense,"[internships, assignments, learning, intense, ...",[ Yeah it was intense! But I had to pause half...,internships - assignments - learning - intense...,0.0,False
8,I should probably start looking too. I’m just...,-1,-1_internships_assignments_learning_intense,"[internships, assignments, learning, intense, ...",[ Yeah it was intense! But I had to pause half...,internships - assignments - learning - intense...,0.0,False
9,Same here. Between school shows and job appli...,-1,-1_internships_assignments_learning_intense,"[internships, assignments, learning, intense, ...",[ Yeah it was intense! But I had to pause half...,internships - assignments - learning - intense...,0.0,False


# Topic Distribution per Chat

In [147]:
from sklearn.metrics.pairwise import cosine_distances
import warnings
warnings.filterwarnings('ignore')

def new_ID(docs_df):
    docs = docs_df['message'].to_list()
    representation_model = KeyBERTInspired()
    topic_model = BERTopic(representation_model=KeyBERTInspired(), min_topic_size=2)
    topic_model = topic_model.fit(docs)
    # print("Conversation:", docs_df["conversation_num"].iloc[0], len(topic_model.get_topic_info()))
    counter = 0
    while (len(topic_model.get_topic_info()) == 1 and counter <= 5):
        counter += 1
        topic_model = topic_model.fit(docs)
        # print("Retrying:", len(topic_model.get_topic_info()))

    if len(topic_model.get_topic_info()) == 1:
        return 0
    
    # TODO: This line FAILS when the number of topics is 1
    topic_distr, _ = topic_model.approximate_distribution(docs)

    mean_vector = np.mean(topic_distr, axis=0)

    info_diversity = np.sum(cosine_distances(topic_distr, mean_vector.reshape(1, -1)))
    info_diversity /= len(topic_distr)
    return info_diversity

def overhead_function(docs):
    # print("Conversation number", docs["conversation_num"])
    docs = docs['message'].to_list()
    return new_ID(docs)

# iterate through conversations, uniquely identified by "conversation_num" parameter
# group by conversation_num


In [148]:
warnings.filterwarnings('ignore')
for i in range(0, 10):
    print("Attempt Number:", i)
    output_df = df.groupby("conversation_num").apply(lambda x: new_ID(x)).to_frame().reset_index()
    output_df.columns = ["conversation_num", "info_diversity"]

    batches = get_batches(output_df)

    for batch in batches:
        feature = "info_diversity"
        
        og_result = batch[0][1][feature]
        inv_result = batch[1][1][feature]
        dir_result = batch[2][1][feature]

        inv_distance = abs(og_result - inv_result)
        dir_distance = abs(og_result - dir_result)

        if dir_distance < inv_distance:
            print("[FAILED] Test Case #", batch[0][1]["conversation_num"], ":", dir_distance / inv_distance)

Attempt Number: 0
Attempt Number: 1
Attempt Number: 2
[FAILED] Test Case # conversation_4 : 0.9255120808110898
Attempt Number: 3
[FAILED] Test Case # conversation_4 : 0.1524170510975725
Attempt Number: 4
Attempt Number: 5
Attempt Number: 6
[FAILED] Test Case # conversation_4 : 0.8195466395333789
Attempt Number: 7
[FAILED] Test Case # conversation_4 : 0.7609662129271485
Attempt Number: 8
[FAILED] Test Case # conversation_4 : 0.9477367878580975
Attempt Number: 9


In [219]:
df.groupby("conversation_num").apply(lambda x: print(x['conversation_num'].iloc[0]))

conversation_1
conversation_2
conversation_3
conversation_4
conversation_5
conversation_6
conversation_7
conversation_8
conversation_9


/var/folders/q6/rj1_hcnn4l1czvx3phrf8kbw0000gn/T/ipykernel_42521/3743902667.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



""


In [104]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,I’ve been thinking about starting a garden bu...,0,0_gardening_garden_vegetables_plants,"[gardening, garden, vegetables, plants, grow, ...",[ I hear you I’m also limited on space but I’m...,gardening - garden - vegetables - plants - gro...,1.000000,False
1,Gardening is super relaxing You should start ...,0,0_gardening_garden_vegetables_plants,"[gardening, garden, vegetables, plants, grow, ...",[ I hear you I’m also limited on space but I’m...,gardening - garden - vegetables - plants - gro...,0.937373,False
2,That sounds fun I’ve always wanted to grow my...,0,0_gardening_garden_vegetables_plants,"[gardening, garden, vegetables, plants, grow, ...",[ I hear you I’m also limited on space but I’m...,gardening - garden - vegetables - plants - gro...,0.985573,True
3,I hear you I’m also limited on space but I’m ...,0,0_gardening_garden_vegetables_plants,"[gardening, garden, vegetables, plants, grow, ...",[ I hear you I’m also limited on space but I’m...,gardening - garden - vegetables - plants - gro...,1.000000,True
4,That’s a great idea You can grow a lot in con...,0,0_gardening_garden_vegetables_plants,"[gardening, garden, vegetables, plants, grow, ...",[ I hear you I’m also limited on space but I’m...,gardening - garden - vegetables - plants - gro...,0.817586,True
5,Speaking of strategy has anyone been followin...,1,1_chess_strategy_play_tournaments,"[chess, strategy, play, tournaments, intense, ...",[ I’m not a chess expert but I enjoy watching ...,chess - strategy - play - tournaments - intens...,1.000000,True
6,I have The level of play this year has been i...,1,1_chess_strategy_play_tournaments,"[chess, strategy, play, tournaments, intense, ...",[ I’m not a chess expert but I enjoy watching ...,chess - strategy - play - tournaments - intens...,0.848621,True
7,I’m not a chess expert but I enjoy watching t...,1,1_chess_strategy_play_tournaments,"[chess, strategy, play, tournaments, intense, ...",[ I’m not a chess expert but I enjoy watching ...,chess - strategy - play - tournaments - intens...,1.000000,True
8,It’s amazing how much thought goes into each ...,1,1_chess_strategy_play_tournaments,"[chess, strategy, play, tournaments, intense, ...",[ I’m not a chess expert but I enjoy watching ...,chess - strategy - play - tournaments - intens...,0.848621,False
9,Definitely I find that playing chess really s...,1,1_chess_strategy_play_tournaments,"[chess, strategy, play, tournaments, intense, ...",[ I’m not a chess expert but I enjoy watching ...,chess - strategy - play - tournaments - intens...,1.000000,False
